# Install packages

In [ ]:
!pip install -Uq emoji \
                 scikit-learn \
                 underthesea \
                 tqdm \
                 flashtext

# Read data

In [ ]:
import numpy as np
import pandas as pd

root = 'https://raw.githubusercontent.com/thinhntr/absa/main/data/csv/'
train_url = root + 'train.csv'
dev_url = root + 'dev.csv'
test_url = root + 'test.csv'


def read_csv(url):
    df = pd.read_csv(url)

    X = df.pop('review')
    y = df.replace({
        np.nan: 0, 
        'negative': 1, 
        'neutral': 2, 
        'positive': 3}) \
        .astype(np.uint8)

    print('X.shape:', X.shape, 'y.shape:', y.shape)
    return X, y

X_train, y_train = read_csv(train_url)
X_dev, y_dev = read_csv(dev_url)
X_test, y_test = read_csv(test_url)

X.shape: (2961,) y.shape: (2961, 12)
X.shape: (1290,) y.shape: (1290, 12)
X.shape: (500,) y.shape: (500, 12)



# EDA

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

y_traincp = y_train.copy()
y_devcp = y_dev.copy()
aspects = y_traincp.columns

In [ ]:
ap_train = (y_traincp != 0).sum(axis=0).sort_values()
ap_dev   = (y_devcp   != 0).sum(axis=0).sort_values()

fig = go.Figure(data=[
    go.Bar(name='train', x=ap_train.index, y=ap_train),
    go.Bar(name='dev'  , x=ap_dev.index  , y=ap_dev)
])
fig

In [ ]:
sentiments = ['-', 'o', '+']

apstm_train = pd.DataFrame()
apstm_dev   = pd.DataFrame()

for aspect in aspects:
    for sentiment_i, sentiment in enumerate(sentiments, 1):
        new_col = f'{aspect}, {sentiment}'
        apstm_train[new_col] = y_traincp[aspect] == sentiment_i
        apstm_dev[new_col]   = y_devcp[aspect]   == sentiment_i


apstm_train = apstm_train.sum(axis=0).sort_values()
apstm_dev   = apstm_dev  .sum(axis=0).sort_values()

fig = go.Figure([
           go.Bar(name='train', x=apstm_train.index, y=apstm_train),
           go.Bar(name='dev'  , x=apstm_dev.index  , y=apstm_dev)
])
fig.update_layout(xaxis_tickangle=90)
fig

# Preprocessing

## Text Cleanup

In [ ]:
import re
import emoji
from flashtext import KeywordProcessor
from sklearn.base import BaseEstimator, TransformerMixin


class TextCleanup(BaseEstimator, TransformerMixin):
    def __init__(self, num_step=range(5)):
        super().__init__()
        self.num_step = num_step

        self.re_emoji = emoji.get_emoji_regexp()
        self.re_hagtag = re.compile('#\S+')
        self.re_giatien = re.compile('((?:(?:\d+[,\.]?)+) ?(?:k|vnd|d|đ|củ))')
        self.re_special_chars = re.compile('[@!#$&^*%<>?/=+`]')

        self.kp = KeywordProcessor(case_sensitive=True)
        rules = {
            "òa": ["oà"], "óa": ["oá"], "ỏa": ["oả"], "õa": ["oã"], "ọa": ["oạ"],
            "òe": ["oè"], "óe": ["oé"], "ỏe": ["oẻ"], "õe": ["oẽ"], "ọe":["oẹ"],
            "ùy": ["uỳ"], "úy": ["uý"], "ủy": ["uỷ"], "ũy": ["uỹ"], "ụy":["uỵ"],
            "ùa": ["uà"], "úa": ["uá"], "ủa": ["uả"], "ũa": ["uã"], "ụa":["uạ"],
            "không": ["k", "K", "hong", "ko", "Ko", "Khong", "khong"],
            "công ty": ["công ti"],
            "lý": ["lí"],
            "xảy": ["xẩy"],
            "bảy": ["bẩy"],
            "gãy": ["gẫy"]
        }
        self.kp.add_keywords_from_dict(rules)

    def step0(self, text):
        """Thay #lozi, #blabla thành hag_tag."""
        return self.re_hagtag.sub('hag_tag', text)

    def step1(self, text):
        """Thay 100k, 200d thành ' giá_tiền '."""
        return self.re_giatien.sub('giá_tiền', text)

    def step2(self, text):
        """Xóa kí tự đặc biệt."""
        return self.re_special_chars.sub('', text)

    def step3(self, text):
        """Xóa emoji 🍿."""
        return self.re_emoji.sub('', text)

    def step4(self, text):
        """Sửa lỗi chính tả

        Đổi các từ phủ định sau: khong, ko, k… thành 'không'.
        Sửa oà, uý thành òa, úy
        """
        return self.kp.replace_keywords(text)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        result = np.empty_like(X)

        steps = np.array([self.step0,
                          self.step1,
                          self.step2,
                          self.step3,
                          self.step4])

        for i, text in enumerate(X):
            for step in steps[self.num_step]:
                text = step(text)
            result[i] = text

        return result


texts = ['K khí trong lành. đồ ăn hong ngon, thức uống  K tồi; 🥙🌮',
         'khung cảnh xinh đẹp',
         '200k quá mắc',
         'món ăn này mắc quá tới 200k lận. ngày 23/3/2000 😴',
         'mua 100.000vnd',
         'bán 1,000,000 d. 5 cái bành xèo tốn 500k',
         'bán 1.000.000 d. 5 cái bành xèo tốn 500k %^^4',
         'món ăn này có giá 10 lít',
         'món ăn này tận 1 củ',
         'bán 1.000đ',
         'quán này có giá trung bình từ 100k-200k 😛',
         'quán này có giá trung bình từ 100-200k 😫',
         '#mắc #food',
         'bàn ghế sạch đẹp, thái độ nhân viên ok#restaurant 😍',
         '#tiktok ở nhà vẫn vui',
         '# birthday ngày mai có tiệc ^^',
         'aslkdhlakd#tiktok#learn asljdalskjd',
         '#tiktok   #learn',
         '#hastag alskjdlasjd #hastag asdsadas #hastag 😁',
         '#123&456',
         '#!?@!']

cleaner = TextCleanup()
cleaner.fit_transform(texts)

array(['không khí trong lành. đồ ăn không ngon, thức uống  khôn',
       'khung cảnh xinh đẹp', 'giá_tiền quá mắc',
       'món ăn này mắc quá tới giá_tiền lận. ngày 2332000 ',
       'mua giá_tiền', 'bán giá_tiền. 5 cái bành xèo tốn giá_tiền',
       'bán giá_tiền. 5 cái bành xèo tốn giá_tiền 4',
       'món ăn này có giá 10 lít', 'món ăn này tận giá_tiền',
       'bán giá_tiền', 'quán này có giá trung bình từ giá_tiền-giá_tiền ',
       'quán này có giá trung bình từ 100-giá_tiền ', 'hag_tag hag_tag',
       'bàn ghế sạch đẹp, thái độ nhân viên okhag_tag ',
       'hag_tag ở nhà vẫn vui', ' birthday ngày mai có tiệc ',
       'aslkdhlakdhag_tag asljdalskjd', 'hag_tag   hag_tag',
       'hag_tag alskjdlasjd hag_tag asdsadas hag_tag ', 'hag_tag',
       'hag_tag'], dtype='<U55')

# Models

In [ ]:
from underthesea import word_tokenize, pos_tag

from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Base Random Forest
model0 = make_pipeline(TextCleanup(range(4)),
                       TfidfVectorizer(),
                       RandomForestClassifier(random_state=18))


# Random Forest
model1 = make_pipeline(TextCleanup(),
                       TfidfVectorizer(ngram_range=(1, 3)),
                       RandomForestClassifier(random_state=18))


model2 = make_pipeline(TextCleanup(),
                       TfidfVectorizer(analyzer=word_tokenize),
                       RandomForestClassifier(random_state=18))


model3 = make_pipeline(TextCleanup(),
                       TfidfVectorizer(tokenizer=word_tokenize,
                                         ngram_range=(1, 2)),
                       RandomForestClassifier(random_state=18))


# SVM
model4 = make_pipeline(TextCleanup(),
                       TfidfVectorizer(ngram_range=(1, 3)),
                       MultiOutputClassifier(LinearSVC(random_state=18)))


model5 = make_pipeline(TextCleanup(),
                       TfidfVectorizer(analyzer=word_tokenize),
                       MultiOutputClassifier(LinearSVC(random_state=18)))


model6 = make_pipeline(TextCleanup(),
                       TfidfVectorizer(tokenizer=word_tokenize,
                                         ngram_range=(1, 2)),
                       MultiOutputClassifier(LinearSVC(random_state=18)))


model7 = make_pipeline(TextCleanup(),
                       TfidfVectorizer(ngram_range=(1, 3),
                                         min_df=2, max_df=0.8,
                                         max_features=1500),
                       MultiOutputClassifier(LinearSVC(random_state=18)))

model8 = make_pipeline(TextCleanup(),
                       TfidfVectorizer(analyzer=pos_tag,
                                       min_df=2, max_df=0.9,
                                       max_features=1200),
                       MultiOutputClassifier(LinearSVC(random_state=18)))


models = (model0, model1, model2, model3, model4, model5, model6, model7, model8)

In [ ]:
from sklearn.pipeline import make_union
from sklearn.multioutput

In [ ]:
class MultiStage(BaseEstimator, ClassifierMixin):
    def __init__(self):
        super().__init__()
        
        self.phaseA = make_pipeline(
            TextCleanup(),
            make_union(TfifdVectorizer(),
                       TfidfVectorizer()),
            ClassifierChain(LinearSVC(random_state=5))
        )
        self.phaseB = 0

    def fit(self, X, y):
        y_phaseA = y != 0
        self.phaseA.fit(X, y_phaseA)
        self.phaseB.fit(X, y_phaseB)
        return self

    def predict(self, X):
        self.phaseA.predict(X)
        return self.phaseB.predict(X)

# Evaluation

## Custom Evaluation Tool

In [ ]:
from tqdm.autonotebook import tqdm
from sklearn.metrics import f1_score
from sklearn.utils.validation import check_is_fitted


def multioutput_to_multilabel(y):
    if isinstance(y, pd.DataFrame): 
        y = y.values
    nrow = y.shape[0]
    ncol = y.shape[1]
    multilabel = np.zeros((nrow, 3 * ncol), dtype=bool)
    for i in range(nrow):
        for j in range(ncol):
            if y[i, j] != 0:
                pos = j * 3 + (y[i, j] - 1)
                multilabel[i, pos] = True
    return multilabel


def evaluate_full_models(X_train, y_train, X_test, y_test,
                         models, average='micro', **kwargs):
    y_test = multioutput_to_multilabel(y_test)
    result = dict()

    for i, model in enumerate(tqdm(models)):
        try: check_is_fitted(model)
        except: model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        y_pred = multioutput_to_multilabel(y_pred)

        score = f1_score(y_test, y_pred, average=average, **kwargs)
        result[f'model{i}'] = round(score*100, 2)

    return pd.Series(result)

### Evaluate on dev

In [ ]:
result_micro = evaluate_full_models(X_train, y_train, X_dev, y_dev, models)
result_micro

  0%|          | 0/9 [00:00<?, ?it/s]

model0    59.84
model1    56.63
model2    57.04
model3    56.41
model4    65.83
model5    64.52
model6    65.69
model7    66.25
model8    64.14
dtype: float64

In [ ]:
result_macro = evaluate_full_models(X_train, y_train, X_dev, y_dev, models, 
                                    average='macro', zero_division=0)
result_macro

  0%|          | 0/9 [00:00<?, ?it/s]

model0    12.63
model1    10.97
model2    10.92
model3    10.76
model4    19.63
model5    21.70
model6    20.63
model7    25.56
model8    24.26
dtype: float64

### Evaluate on test

## Official Evaluation Tool

In [ ]:
import json


sentiments = [None, 'negative', 'neutral', 'positive']

with open(data_dir/'aspects.json') as f:
    aspects = json.load(f)


def label_decoder(encoded_label):
    label = []
    for ap_idx, stm_idx in enumerate(encoded_label):
        if stm_idx != 0:
            aspect = aspects[ap_idx]
            sentiment = sentiments[stm_idx]
            label.append(f'{{{aspect}, {sentiment}}}')

    return ', '.join(label)


def save_result(X, y, save_path):
    rows = []
    for test_id, (review, encoded_label) in enumerate(zip(X, y), 1):
        label = label_decoder(encoded_label)
        rows.extend((f'#{test_id}', review, label, ''))

    text = '\n'.join(rows[:-1])
    with open(save_path, mode='w', encoding='utf-8-sig') as output_file:
        output_file.write(text)

    
def evaluate(model, X, y_true_path, y_pred_path):
    y = model.predict(X)
    save_result(X, y, y_pred_path)
    !java SAEvaluate.java {y_true_path} {y_pred_path}

### Evaluate on dev

In [ ]:
evaluate(model0, X_dev, "y_dev.txt", "y_pred_model0.txt")

In [ ]:
evaluate(model1, X_dev, "y_dev.txt", "y_pred_model1.txt")

In [ ]:
evaluate(model2, X_dev, "y_dev.txt", "y_pred_model2.txt")

### Evaluate on test

# Draft